In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import os
import time


# 1. Install torchdiffeq
# (Run this in your terminal)
!pip install torchdiffeq

from torchdiffeq import odeint

In [19]:
# Set the Configuration

# File paths
NPZ_FILE = 'training_data.npz'
PARAMS_FILE = 'normalization_params.json'
MODEL_SAVE_PATH = 'chem_node_net.pth'

# Training hyperparameters
N_EPOCHS = 2000
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
HIDDEN_DIMS = 256  

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



# -----------------------------------------------------------------
# Load data to get model dimensions

try:
    with np.load(NPZ_FILE) as data:
        all_trajectories = data['trajectories']
        t_points = data['times']
    
    # Get the number of features (e.g., 22 for your ammonia mech)
    # Shape is (num_sims, num_timesteps, num_features)
    N_FEATURES = all_trajectories.shape[2]
    
    print(f"Data loaded. Found {N_FEATURES} features (species + temp).")

except FileNotFoundError:
    print(f"Error: Could not find '{NPZ_FILE}'.")
    print("Please make sure you have run Phase 2.")
    raise

Using device: cpu
Data loaded. Found 32 features (species + temp).


In [20]:
# -----------------------------------------------------------------
# Step 1: The Derivative Network 

class ChemNet(nn.Module):
    def __init__(self, input_features, hidden_features):
        super(ChemNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ELU(),  # Exponential Linear Unit [cite: 866]
            nn.Linear(hidden_features, hidden_features),
            nn.ELU(),
            nn.Linear(hidden_features, hidden_features),
            nn.ELU(),
            nn.Linear(hidden_features, input_features)
        )

    def forward(self, t, y):
        # The 'odeint' solver requires the network's forward
        # method to accept (time, state) as arguments.
        # We don't use 't' here, but it's a required argument.
        return self.net(y)

In [21]:
# Step 2: The Neural ODE (NODE) Wrapper
# -----------------------------------------------------------------
# This module wraps our ChemNet and calls the 'odeint' solver.

class NeuralODE(nn.Module):
    def __init__(self, derivative_net):
        super(NeuralODE, self).__init__()
        self.net = derivative_net

    def forward(self, y0, t_points):
        # y0 shape: (batch_size, n_features)
        # t_points shape: (n_timesteps,)
        
        # 'odeint' solves the ODE defined by self.net
        # 'method='dopri5'' is a standard Runge-Kutta solver, 
        # as suggested by the Bansude paper (they used 'dopri5' for training)
        # 'adjoint' method is used for efficient backpropagation 
        pred_y = odeint(self.net, y0, t_points, method='dopri5')
        
        # Output shape from odeint is (n_timesteps, batch_size, n_features)
        return pred_y

In [22]:
# Step 3: The PyTorch Dataset and DataLoader

# We create a simple Dataset to feed trajectories to our model.
class ODESolutionDataset(Dataset):
    def __init__(self, trajectories, device):
        # Data shape: (num_sims, n_timesteps, n_features)
        # We convert to float32 (standard for NN) and move to device
        self.data = torch.tensor(trajectories, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Returns one full trajectory
        # Shape: (n_timesteps, n_features)
        return self.data[idx]


# --- Prepare data for training ---

# Convert time points to a tensor on the correct device
t_points = torch.tensor(t_points, dtype=torch.float32).to(device)

# Create Dataset and DataLoader
dataset = ODESolutionDataset(all_trajectories, device)
# The DataLoader will batch trajectories
# A 'batch' will have shape: (BATCH_SIZE, n_timesteps, n_features)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [23]:
# Step 4: The Training Loop

# 1. Initialize models
derivative_net = ChemNet(N_FEATURES, HIDDEN_DIMS).to(device)
model = NeuralODE(derivative_net).to(device)

# 2. Loss Function and Optimizer
loss_function = nn.L1Loss()  # L1Loss is MAE

# --- FIX 1: Define Optimizer BEFORE Scheduler ---
# Bansude et al. used Adam 
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# This will reduce the LR by half if the loss doesn't improve for 50 epochs
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    'min', 
    factor=0.5, 
    patience=50, 
    verbose=True)

In [ ]:
print(f"--- Starting Training on {device} ---")

# --- Start training ---
start_time = time.time()
for epoch in range(N_EPOCHS):
    epoch_loss = 0.0
    
    # Set model to training mode
    model.train()
    for batch_y_true in data_loader:
        # batch_y_true shape: (BATCH_SIZE, n_timesteps, n_features)
        
        # Get the initial condition (the state at t=0)
        # y0 shape: (BATCH_SIZE, n_features)
        batch_y0 = batch_y_true[:, 0, :]
        
        # --- Forward Pass ---
        # Run the Neural ODE
        # pred_y shape: (n_timesteps, BATCH_SIZE, n_features)
        pred_y = model(batch_y0, t_points)
        
        # --- Calculate Loss ---
        # We must re-order pred_y to match batch_y_true
        # [timesteps, batch, features] -> [batch, timesteps, features]
        pred_y_for_loss = pred_y.permute(1, 0, 2)
        
        loss = loss_function(pred_y_for_loss, batch_y_true)

        # --- Backward Pass ---
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(data_loader)
    
    # --- FIX 2: Call the scheduler.step() at the end of each epoch ---
    scheduler.step(avg_loss)
    # -----------------------------------------------------------------

    # Updated print statement to show the learning rate
    if (epoch + 1) % 10 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch + 1}/{N_EPOCHS} | Avg. Loss: {avg_loss:.6f} | LR: {current_lr:.2e}")

end_time = time.time()
print(f"--- Training Complete ---")
print(f"Total time: {(end_time - start_time):.2f} seconds")

--- Starting Training on cpu ---
Epoch 10/2000 | Avg. Loss: 0.055133 | LR: 1.00e-03
Epoch 20/2000 | Avg. Loss: 0.053256 | LR: 1.00e-03
Epoch 30/2000 | Avg. Loss: 0.050271 | LR: 1.00e-03
Epoch 40/2000 | Avg. Loss: 0.052117 | LR: 1.00e-03
Epoch 50/2000 | Avg. Loss: 0.050759 | LR: 1.00e-03
Epoch 60/2000 | Avg. Loss: 0.045410 | LR: 1.00e-03
Epoch 70/2000 | Avg. Loss: 0.043345 | LR: 1.00e-03
Epoch 80/2000 | Avg. Loss: 0.042743 | LR: 1.00e-03
Epoch 90/2000 | Avg. Loss: 0.044099 | LR: 1.00e-03
Epoch 100/2000 | Avg. Loss: 0.043567 | LR: 1.00e-03
Epoch 110/2000 | Avg. Loss: 0.043214 | LR: 1.00e-03
Epoch 120/2000 | Avg. Loss: 0.038296 | LR: 1.00e-03
Epoch 130/2000 | Avg. Loss: 0.037354 | LR: 1.00e-03
Epoch 140/2000 | Avg. Loss: 0.036262 | LR: 1.00e-03
Epoch 150/2000 | Avg. Loss: 0.035840 | LR: 1.00e-03
Epoch 160/2000 | Avg. Loss: 0.032348 | LR: 1.00e-03
Epoch 170/2000 | Avg. Loss: 0.032558 | LR: 1.00e-03
Epoch 180/2000 | Avg. Loss: 0.028601 | LR: 1.00e-03
Epoch 190/2000 | Avg. Loss: 0.029126 | L

In [15]:
# Step 5: Save the Trained Model

# We only need to save the derivative network (ChemNet),
# as it *is* the learned dynamics function.
torch.save(derivative_net.state_dict(), MODEL_SAVE_PATH)
print(f"Trained model saved to: {MODEL_SAVE_PATH}")

Trained model saved to: chem_node_net.pth
